In [1]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq


# %% [markdown]
# (If you have import errors, try restarting your Jupyter kernel)
# 

# %% [markdown]
# ### Load dataset
# 

# %%
from datasets import load_dataset
from datasets import load_dataset
train_dataset = load_dataset('json', data_files='/root/finetune-llm-for-rag/datasets/sql/MiniLM-L6/sql-create-context-spider-intersect-train-with-prompts.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='/root/finetune-llm-for-rag/datasets/sql/MiniLM-L6/sql-create-context-spider-intersect-validation-with-prompts.jsonl', split='train')
print(train_dataset)
print(eval_dataset)

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-451703fd23ff8620/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-941f6b50a392c31c/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 3961
})
Dataset({
    features: ['db_id', 'context', 'question', 'answer', 'full_prompt', 'inference_prompt'],
    num_rows: 568
})


In [2]:
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")


In [11]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

# %% [markdown]
# Setup the tokenize function to make labels and input_ids the same. This is basically what [self-supervised fine-tuning](https://neptune.ai/blog/self-supervised-learning) is:

# %%
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )
    # print("result is: ", result)
    print("length of input ids: ", len(result["input_ids"]))

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

# %% [markdown]
# And run convert each data_point into a prompt that I found online that works quite well:

# %%
def generate_and_tokenize_prompt(data_point):
    full_prompt = data_point["full_prompt"]
    # print(full_prompt)
    return tokenize(full_prompt)

In [12]:
# tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

length of input ids:  426
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  469
length of input ids:  512
length of input ids:  505
length of input ids:  484
length of input ids:  481
length of input ids:  512
length of input ids:  452
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  447
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  511
length of input ids:  434
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  422
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of input ids:  512
length of in

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
# Add EOS token and padding configurations
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

# Initialize variables



The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
print(tokenizer.bos_token)
print(tokenizer.eos_token)

<s>
</s>


In [15]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=True,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

In [26]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

# Tokenize a short prompt with padding and max_length set to 10
prompt = "Hello saudfjapwejsdjfioap  osdifwe89jaf asoidfa ssa p apfsdj oeia spda p apapap aifoias dfda ddoao sss aaooa "
result = tokenizer(
    prompt,
    truncation=True,
    max_length=5,
    padding=True,
    return_tensors=None,
)

# Display the tokenized IDs and check for padding token ID
print("Tokenized IDs:", result['input_ids'])
print("Pad Token ID:", tokenizer.pad_token_id)
print("length of input ids: ", len(result["input_ids"]))
# Check if pad token ID is present in the tokenized sequence
if tokenizer.pad_token_id in result['input_ids']:
    print("Pad token ID is present in the tokenized sequence.")
else:
    print("Pad token ID is not present in the tokenized sequence.")

Tokenized IDs: [1, 15043, 872, 566, 2]
Pad Token ID: 0
length of input ids:  5
Pad token ID is not present in the tokenized sequence.


In [16]:
tokenized_text = tokenize("hello mate")
print(tokenized_text)

{'input_ids': [1, 22172, 15358, 2], 'attention_mask': [1, 1, 1, 1], 'labels': [1, 22172, 15358, 2]}


In [7]:
# Sam's manual tests. We want to just investigate the different properties of tokenizers and what they do.
tokenized_text = tokenizer("hello mate", return_tensors="pt") # , padding='max_length', max_length=max_input_tokens+10
print(tokenized_text)

{'input_ids': tensor([[    1, 22172, 15358,     2]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [4]:
base_model = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
base_text = "Add more text here to increase the token count. " * 100

max_new_tokens = 5
max_input_tokens = 0
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# Add EOS token and padding configurations
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"
# tokenizer.padding_size = "left"
while True:
    batch_model_inputs = tokenizer(base_text, return_tensors="pt", padding='max_length', max_length=max_input_tokens+10)
    batch_model_inputs.to("cuda")
    input_tokens = batch_model_inputs['input_ids'].shape[1]
    print("input tokens: ", input_tokens)
    if input_tokens > max_input_tokens:
        max_input_tokens = input_tokens
    
    output = model.generate(**batch_model_inputs, max_new_tokens=max_new_tokens)
    output_tokens = output.shape[1]
    # Check if the last tokens of the input are present in the output
    last_input_tokens = batch_model_inputs['input_ids'][0, -max_new_tokens:].tolist()
    if not all(token in output[0, -max_new_tokens*2:].tolist() for token in last_input_tokens):
        print(f"Context limit reached with input tokens: {input_tokens}")
        break

    base_text += " Add more text here to increase the token count. "*30


input tokens:  1003
input tokens:  1333
input tokens:  1663
input tokens:  1993
input tokens:  2323
input tokens:  2653
input tokens:  2983
input tokens:  3313
input tokens:  3643
input tokens:  3973
input tokens:  4303


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


input tokens:  4633
input tokens:  4963
input tokens:  5293
input tokens:  5623
input tokens:  5953


KeyboardInterrupt: 